# Случайный лес

АЛГОРИТМ ПОСТРОЕНИЯ СЛУЧАЙНОГО ЛЕСА, СОСТОЯЩЕГО ИЗ ДЕРЕВЬЕВ

Для каждого:

- сгенерировать выборку с помощью бутстрэпа;
- построить решающее дерево по выборке: по заданному критерию мы выбираем лучший признак, делаем разбиение в дереве по нему и так до исчерпания выборки → дерево строится, пока в каждом листе не более объектов или пока не достигнем определенной высоты дерева → при каждом разбиении сначала выбирается несколько случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них.

Итоговый классификатор $a(x) = \frac{1}{N} \sum_{i=1}^{N}{b_{i} (x)}$, иными словами — для задачи классификации мы выбираем решение голосованием по большинству, а в задаче регрессии — средним.

Рекомендуется в задачах классификации брать $m = \sqrt{(n)}$, а в задачах регрессии — $m = \frac{n}{3}$, где n — число признаков. Также рекомендуется в задачах классификации строить каждое дерево до тех пор, пока в каждом листе не окажется по одному объекту, а в задачах регрессии — пока в каждом листе не окажется по пять объектов.

Таким образом, случайный лес — это бэггинг над решающими деревьями, при обучении которых для каждого разбиения признаки выбираются из некоторого случайного подмножества признаков.

## РЕАЛИЗАЦИЯ НА PYTHON  И ПОДБОР ПАРАМЕТРОВ

Теперь обучим случайный лес на простых данных и посмотрим, как можно подбирать параметры случайного леса для достижения наилучшего качества модели.

> Потренируемся на данных, по которым мы будем предсказывать погоду. Датасет можно найти [здесь temps_extended.csv](./../data/temps_extended.csv).

Откроем его, удалим признаки, не относящиеся к предсказанию (от дня недели, например, или от года погода не зависит), разделим на тестовую и обучающуюся выборки:

In [1]:
import pandas as pd
import numpy as np

from sklearn import model_selection, datasets, metrics, tree, ensemble, linear_model

In [2]:
weather = pd.read_csv('./../data/temps_extended.csv')
y = weather['actual']
X = weather.drop(['actual', 'weekday', 'month', 'day', 'year'], axis=1)
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.3, random_state=42)

Попробуем подобрать гиперпараметры таким образом, чтобы получить оптимальный результат.

Если мы запускаем случайный лес без настройки параметров, то по умолчанию они следующие:

In [3]:
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint

rf = RandomForestRegressor(random_state=42)
# Look at parameters used by our current forest
print('Параметры по умолчанию:\n')
pprint(rf.get_params())

Параметры по умолчанию:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


Попробуем подбирать разные значения для некоторых параметров. Для перебора вариантов возьмем следующие параметры:

- n_estimators 
- max_features 
- max_depth 
- min_samples_split 
- min_samples_leaf
- bootstrap

Мы можем сами указать, какие значения гиперпараметров надо перебрать.

Зададим сетку гиперпараметров, которые будут перебираться:

In [4]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

Обучим наш лес:

In [5]:
rf = RandomForestRegressor(random_state=42)
rf_random = RandomizedSearchCV(
    estimator=rf,
    param_distributions=random_grid,
    n_iter=100,
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

Давайте посмотрим, какие гиперпараметры нам предлагают как оптимальные:

In [6]:
rf_random.best_params_

{'n_estimators': 1000,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': True}

## Задача 1

Обучите случайный лес с предустановленными параметрами и теми параметрами, которые мы отобрали как оптимальные. В обоих вариантах поставьте `random_state=42`. Какое улучшение MSE дала подстановка отобранных гиперпараметров? Ответ округлите до одного знака после запятой.

In [7]:
np.round(metrics.mean_squared_error(y_test, rf_random.predict(X_test)), 1)

23.1

In [12]:
rf2 = RandomForestRegressor(random_state=42)
rf2.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [14]:
np.round(metrics.mean_squared_error(y_test, rf2.predict(X_test)) -
         metrics.mean_squared_error(y_test, rf_random.predict(X_test)), 1)

1.5

## Практика

- Загрузите датасет digits с помощью функции **load_digits** из **sklearn.datasets** и подготовьте матрицу признаков **X** и ответы на обучающей выборке **y** (вам потребуются поля data и target в объекте, который возвращает load_digits). 

- Информацию о датасете вы можете получить, обратившись к полю DESCR у возвращаемого объекта load_digits. Нам предстоит решать задачу классификации изображений с цифрами по численным признакам.

- Для оценки качества мы будем использовать **cross_val_score** из **sklearn.model_selection** с параметром cv=10. Эта функция реализует k-fold cross validation c **k** равным значению параметра **cv**. Предлагается использовать k=10, чтобы полученные оценки качества имели небольшой разброс, и было проще проверить полученные ответы. На практике же часто хватает и *k=5*. Функция cross_val_score будет возвращать numpy.ndarray, в котором будет *k* чисел — качество в каждом из *k* экспериментов k-fold cross validation. Для получения среднего значения (которое и будет оценкой качества работы) вызовите метод .mean() у массива, который возвращает cross_val_score.

С небольшой вероятностью вы можете натолкнуться на случай, когда полученное вами качество в каком-то из пунктов не попадёт в диапазон, заданный для правильных ответов — в этом случае попробуйте перезапустить ячейку с cross_val_score несколько раз и выбрать наиболее «типичное» значение. Если это не помогает, то где-то была допущена ошибка.

Чтобы ускорить вычисление cross_val_score, следует попробовать использовать параметр n_jobs. Число, которое вы подаёте в качестве этого параметра, соответствует количеству потоков вашего процессора, которое будет задействовано в вычислении. Если указать n_jobs = -1, тогда будут задействовано максимальное число потоков.

In [55]:
from sklearn.datasets import load_digits

In [57]:
digits_df = load_digits()

In [62]:
print(digits_df.DESCR)

.. _digits_dataset:

Optical recognition of handwritten digits dataset
--------------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 1797
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each blo

In [63]:
X, y = digits_df['data'], digits_df['target']

In [81]:
def estimate_accuracy(clf, X, y, cv=5):
    return np.round(
        model_selection.cross_val_score(clf, X, y, cv=cv, scoring='accuracy', n_jobs=-1).mean(),
        3
    )

#### Задание 1

Создайте DecisionTreeClassifier с настройками по умолчанию и измерьте качество его работы с помощью cross_val_score.

Эту величину введите в поле для ответа (ваше значение должно попасть в заданный интервал).

In [82]:
des_tree = tree.DecisionTreeClassifier()
des_tree.fit(X, y)

DecisionTreeClassifier()

In [83]:
 estimate_accuracy(des_tree, X, y, 10)

0.82

#### Задание 2

Теперь давайте обучим BaggingClassifier на основе DecisionTreeClassifier. Из sklearn.ensemble импортируйте BaggingClassifier, все параметры задайте по умолчанию. Нужно изменить только количество базовых моделей, задав его равным .

В поле для ответа введите качество бэггинга на нашем датасете (ваше значение должно попасть в заданный интервал).

Подумайте, какие выводы можно сделать из соотношения качества одиночного дерева и бэггинга деревьев?

In [84]:
bagging_clf = ensemble.BaggingClassifier(tree.DecisionTreeClassifier(), n_estimators=100)
bagging_clf.fit(X, y)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100)

In [85]:
estimate_accuracy(bagging_clf, X, y, 10)

0.923

#### Задание 3

Теперь изучите параметры BaggingClassifier и выберите их такими, чтобы каждый базовый алгоритм обучался не на всех d признаках, а на $\sqrt{d}$ случайных признаках.

В поле для ответа введите качество работы получившегося классификатора (ваше значение должно попасть в заданный интервал).

Корень из числа признаков — часто используемая эвристика в задачах классификации, в задачах регрессии же часто берут число признаков, деленное на три, log d тоже имеет место быть. Но в общем случае ничто не мешает вам выбирать любое другое число случайных признаков, добиваясь лучшего качества на кросс-валидации.

In [89]:
features_len = len(digits_df.feature_names)

In [91]:
bagging_clf = ensemble.BaggingClassifier(
    tree.DecisionTreeClassifier(),
    n_estimators=100,
    max_features=int(np.sqrt(features_len))
)

bagging_clf.fit(X, y)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_features=8,
                  n_estimators=100)

In [92]:
estimate_accuracy(bagging_clf, X, y, 10)

0.928

#### Задание 4

В предыдущем пункте мы выбирали подмножество один раз для каждого очередного дерева. Следующим нашим шагом будет построение бэггинга на основе деревьев, которые выбирают случайное подможество признаков для каждой вершины дерева.

Для этого нам потребуется перенести отвечающий за это параметр из BaggingClassifier в DecisionTreeClassifier. Для этого вам из документации нужно выяснить, какой параметр DecisionTreeClassifier за это отвечает.

В поле для ответа введите значение этого параметра (ваше значение должно попасть в заданный интервал).

По-прежнему сэмплируем sqrt(d) признаков.

In [93]:
bagging_clf = ensemble.BaggingClassifier(
    tree.DecisionTreeClassifier(max_features=int(np.sqrt(features_len))),
    n_estimators=100
)

bagging_clf.fit(X, y)

BaggingClassifier(base_estimator=DecisionTreeClassifier(max_features=8),
                  n_estimators=100)

In [94]:
estimate_accuracy(bagging_clf, X, y, 10)

0.951

Полученный в задании 4 классификатор — бэггинг на рандомизированных деревьях (в которых при построении каждой вершины выбирается случайное подмножество признаков и разбиение ищется только по ним). Это в точности соответствует алгоритму Random Forest, поэтому почему бы не сравнить качество работы классификатора с RandomForestClassifier из sklearn.ensemble?

In [113]:
random_forest = ensemble.RandomForestClassifier(
    n_estimators=100,
    n_jobs=-1,
    max_features=int(np.sqrt(features_len)),
    max_depth=30
)

estimate_accuracy(random_forest, X, y, 10)

0.953

- При очень маленьком числе деревьев (5, 10, 15) случайный лес работает хуже, чем при большем числе деревьев.
- С ростом количества деревьев в случайном лесе, в какой-то момент деревьев становится достаточно для высокого качества классификации, а затем качество существенно не меняется.
- При большом количестве признаков (для данного датасета - 40-50) качество классификации становится хуже, чем при малом количестве признаков (10-15). Это связано с тем, что чем меньше признаков выбирается в каждом узле, тем более различными получаются деревья (ведь деревья сильно неустойчивы к изменениям в обучающей выборке), и тем лучше работает их композиция.
- При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса заметно хуже, чем без ограничений, т.к. деревья получаются недообученными. С ростом глубины качество сначала улучшается, а затем не меняется существенно, так как из-за усреднения прогнозов и различий деревьев их переобученность в бэггинге не сказывается на итоговом качестве (все деревья преобучены по-разному, и при усреднении они компенсируют переобученность друг друга).